In [1]:
import tensorflow as tf
import tensorflowjs as tfjs
from matplotlib import pyplot
import numpy as np
import os
import json

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory("data-150x150",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(150, 150),
  batch_size=32)

Found 79696 files belonging to 4 classes.
Using 63757 files for training.


In [3]:
val_ds = tf.keras.utils.image_dataset_from_directory("data-150x150",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(150, 150),
  batch_size=32)

Found 79696 files belonging to 4 classes.
Using 15939 files for validation.


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    #tf.keras.layers.RandomCrop(120, 120) #height, width; NOTE: DOESN'T WORK WITH tensorflowjs
  ]
)

In [6]:
num_classes = 4
models = {}

In [7]:
models["CNN"] = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(150,150,3)),
  tf.keras.layers.Rescaling(1./255),
  data_augmentation,
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [8]:
models["MLP"] = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(150,150,3)),
    tf.keras.layers.Rescaling(1./255),
    data_augmentation,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

In [9]:
for model in models.values():
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
for model in models.values():
    model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 150, 150, 3)       0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)       

In [11]:
epochs=65
histories = {}

In [12]:
for name in models:
    print("Training "+name+":")
    model_path = os.path.join("saved_models", name)
    history_path = os.path.join("saved_histories", name+".json")
    tfjs_target_path = os.path.join("web-client", "saved_models", "layers", name)
    histories[name] = models[name].fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs,
    )
    models[name].save(model_path)
    models[name].save(model_path+".h5")
    tfjs.converters.save_keras_model(models[name], tfjs_target_path)
    json.dump(histories[name].history, open(history_path, 'w'))
    print()

Training CNN:
1993/1993 [==============================] - 70s 29ms/step - loss: 0.8097 - accuracy: 0.6390 - val_loss: 0.4873 - val_accuracy: 0.8175
INFO:tensorflow:Assets written to: saved_models/CNN/assets

Training MLP:
1993/1993 [==============================] - 25s 12ms/step - loss: 1.4783 - accuracy: 0.2636 - val_loss: 1.3862 - val_accuracy: 0.2538
INFO:tensorflow:Assets written to: saved_models/MLP/assets



In [13]:
for name in histories:
    history = histories[name]
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    pyplot.figure(figsize=(8, 8))
    pyplot.subplot(1, 2, 1)
    pyplot.plot(epochs_range, acc, label='Training Accuracy')
    pyplot.plot(epochs_range, val_acc, label='Validation Accuracy')
    pyplot.legend(loc='lower right')
    pyplot.title(name+': Training and Validation Accuracy')

    pyplot.subplot(1, 2, 2)data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAeoAAAHiCAYAAAAnPo9XAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8QVMy6AAAACXBIWXMAAAsTAAALEwEAmpwYAAA44UlEQVR4nO3df5xWdZ3//8dLUEjxBwhmAgZuIoo4/Bih/JGQ2ZIa5K+CtCQ3f23qR/qYWZmy/ths1/3WuqvumplWrmS28sEVM1EJ1x8r+KsEpRApR8sQE3FREXh9/7jOjBfjMDPADHNgHvfb7brNdc55n3O9rmuuc57XOdf7OicyE0mSVE7bdHQBkiRp/QxqSZJKzKCWJKnEDGpJkkrMoJYkqcQMakmSSsyg3kARcVdEnNzWbTtSRCyJiI+3w3JnR8SXivsnRsQvW9N2Ix5nz4h4IyK6bGytUmu5Ddig5boNaAObJagj4nMRMa94If9YvHkPKaZNjYiMiM9Ute9ajBtQDN9YDI+qavOhiGjVj8CLx62/rY2IN6uGT9yQ55KZn8zMm9q6bRlFxAURMaeJ8b0jYlVE7N/aZWXmzZn5iTaqa52NSmb+ITN7ZOaatlh+E48XEbE4Iha0x/I7A7cBWya3AVC87z7U1svdEO0e1BHxFeB7wN8D7wf2BK4BJlQ1exX4uxY+Db0KXLYxNRT/wB6Z2QP4A/CpqnE3V9XadWOWvxX7CXBQRAxsNH4i8JvMfLoDauoIHwV2A/aKiAM35wNvDe9JtwFbNLcBJdCuQR0ROwOXAF/OzP/MzP/NzHcy847M/GpV018Aq4CTmlncTcABEXFYG9Y3JiLqIuJrEfEn4IcR0TMi/isilkbEX4r7/armqT6UMzki/jsirizaPh8Rn9zItgMjYk5ErIiIWRFxdUT8ZD11t6bGSyPiwWJ5v4yI3lXTPx8Rv4+IZRHxzfW9PplZB9wHfL7RpC8AP2qpjkY1T46I/64aPiIino2I5RHxr0BUTfuriLivqO+ViLg5InYppv2Yyob+jmJv6PyIGFB86u1atNkjImZExKsRsSgiTq1a9tSIuDUiflS8NvMjonZ9r0HhZOD/ATOL+9XPa0hE3FM81ssR8Y1ifJeI+EZEPFc8zmMR0b9xrUXbxu+TByPiuxGxDJja3OtRzNM/Iv6z+D8si4h/jYjtipqGVrXbLSJWRkSfFp5vm3Eb4Dag6jG35G1AU89n52IZS4vX8sKI2KaY9qGI+FXx3F6JiJ8W46NYt/8cEa9HxG+iFUcl2nuP+iNAd+D2Ftol8C3g4ojYdj1tVlL5RH55UxOjcojmvzaixt2BXsAHgdOovCY/LIb3BN4E/rWZ+UcDC4HewD8AP4iI2Ii2/wE8CuwKTOW9K0a11tT4OeCLVPYEtwPOA4iI/YBri+XvUTxekytW4abqWiJiH2BYUe+Gvlb1y+gN/CdwIZXX4jng4OomwLeL+vYF+lN5TcjMz7PuHtE/NPEQ04C6Yv7jgb+PiI9VTR9ftNkFmNFczRGxfbGMm4vbxIjYrpi2IzCLSsjsAXwIuLeY9SvAJOBIYCfgFCrv4dYYDSymsvd5Oc28HlHZA/0v4PfAAKAvMC0zVxXPsTr4JgH3ZubSVtbRFtwGtL6t24CqJpRkG9CMfwF2BvYCDqPy4eWLxbRLgV8CPam8tv9SjP8ElSN0g4p5PwMsa/GRMrPdbsCJwJ9aaDMV+Elx/3+AM4GuVFbcAcX4G6kc8upG5R/0SSobxdyImpYAHy/uj6HyKb57M+2HAX+pGp4NfKm4PxlYVDVt+6Lu3TekLZU3+Gpg+6rpP6l/XVrxnJqq8cKq4b8FflHcv4jKhrx+2g7Fa/Dx9Sx7e+B14KBi+HLg/23ka/Xfxf0vAI9UtQsqK9WX1rPcTwNPNPU/LIYHFK9lVyor9Bpgx6rp3wZurHq/zaqath/wZjOv7UnA0mLZ3YHlwDHFtEnVdTWabyEwoYnxDbU28zr9oYX/d8PrQSUIl1Yvr6rdaCrrSxTD84DPbOg6syk33Aa0qi1uA8q8DUjgQ43GdSles/2qxp0OzC7u/wi4DujXaL6PAb8FPgxs09r3bHvvUS8Dekfrv/e5EPgmlQ3ie2Tm21Q+qVzaNuUBsDQz36ofiIjtI+Lfi0MZrwNzgF1i/d+d/amqvvo9ph4b2HYP4NWqcQAvrK/gVtb4p6r7K6tq2qN62Zn5vzTzia6o6WfAF4pP/idSeRNuzGtVr3ENWT0cEe+PiGkR8WKx3J9Q+dTdGvWv5Yqqcb+nsqdZr/Fr072Z9+jJwK2Zubp4n/ycdw9/96eyJ9CU5qa1ZJ3/fQuvR3/g95m5uvFCMvN/qDy/MRExmEqwzdjImjaW24DWtXUbUN5tQFN6A9sWy23qMc6n8uHj0eLQ+ikAmXkflb33q4E/R8R1EbFTSw/W3kH9MPA2lU9DLcrMe4BFVD79rc8PqRyuOHYTa2t42EbD/xfYBxidmTtROUwBVd+ftIM/Ar2Kw6z1+jfTflNq/GP1sovH3LWFeW6icojmCGBH4I5NrKNxDcG6z/fvqfxfhhbLPanRMpvr6fsSlddyx6pxewIvtlDTe0Tlu7aPASdFxJ+i8h3m8cCRxaG7F6gc9mrKC8BfNTH+f4u/1f/r3Ru1afz8mns9XgD2bGYjc1PR/vPAbdWBtJm4DWgdtwEl3AY04xXgHSqH/N/zGJn5p8w8NTP3oLKnfU0UPccz86rMHEllT34Q8FVa0K5BnZnLqRxmuToiPl18+to2Ij4ZEU19rwCVT9PnN7PM1cDFwNfavmKg8iZ8E3gtInoVj9WuMvP3VA5LTo1KJ6CPAJ9qpxpvA46OiEOK71ovoeX3wQPAa1QO5dR//7kpddwJDImIY4uAOYd1w2pH4A1geUT05b1v5JdZT0Bm5gvAQ8C3I6J7RBwA/A2VT+Qb6vNUDlPVfyc3jMqKVUflsPd/AR+IiHMjoltE7BgRo4t5rwcujYi9o+KAiNg1K98Pv0gl/LsUn7SbCvRqzb0ej1LZ6F0RETsUz7n6u76fAMdQ2dD9aCNeg03iNqB13AaUdhtQb7tiWd0jov5oz63A5cV6/0Eq/VJ+AhARJ8S7ner+QuWDxdqIODAiRkelH8b/Am8Ba1t68Hb/eVZm/hOVJ3Ahle/SXgDOAqavp/2DVDY+zbmFysapQVR62N61qfVS+RnJ+6h8YnqESkehzeFEKt83LqPyXdxPqeyJNOV7bGSNmTkf+DKVjiB/pPImqmthnqSykf8g627sN6qOzHwFOAG4gsrz3Rt4sKrJ3wEjqHwffCeVTifVvg1cGBGvRcR5TTzEJCrfWb1EpRPTxZk5qzW1NXIycE3x6bjhBvwbcHJxaO0IKhvUPwG/A8YW8/5/VFbkX1L5fu8HVF4rgFOpbHiWAUOobFSas97XIyu/G/0UlcPaf6Dyv/xs1fQXgMepbCge2PCXYNO5DWg1twHvKss2oN58Kh9I6m9fBM6mEraLgf+m8nreULQ/EPifiHiDytdN/yczF1PpWPp9Kq/576k8939s6cHrO5moZKLSnf/ZzGz3T/PaukXEDcBLmXlhR9ei1nMboHqeQrQkikMifxUR20TEOCong5jewWVpCxeVM3sdS2WPXiXmNkDr41l4ymN3Kod3dqVyGOrMzHyiY0vSliwiLgWmAN/OzOc7uh61yG2AmuShb0mSSsxD35IklZhBLUlSiZXuO+revXvngAEDOroMqfQee+yxVzJzs11gY2O4Pkut09z6XLqgHjBgAPPmzevoMqTSi4jft9yqY7k+S63T3PrsoW9JkkrMoJYkqcQMakmSSqx031FLklr2zjvvUFdXx1tvbe4LomlTdO/enX79+rHtttu2eh6DWpK2QHV1dey4444MGDCAylUiVXaZybJly6irq2PgwIGtns9D35K0BXrrrbfYddddDektSESw6667bvBREINakrZQhvSWZ2P+Zwa1JGmDLVu2jGHDhjFs2DB23313+vbt2zC8atWqZuedN28e55xzTouPcdBBB7VJrbNnz+boo49uk2V1BL+jljqR4vKJ/wx0Aa7PzCsaTf8uMLYY3B7YLTN3KaadDNRf0/qyzLxpsxStUtp111158sknAZg6dSo9evTgvPPOa5i+evVqunZtOmJqa2upra1t8TEeeuihNql1S+cetdRJREQX4Grgk8B+wKSI2K+6TWZOycxhmTkM+Bcql10kInoBFwOjgVHAxRHRczOWry3A5MmTOeOMMxg9ejTnn38+jz76KB/5yEcYPnw4Bx10EAsXLgTW3cOdOnUqp5xyCmPGjGGvvfbiqquualhejx49GtqPGTOG448/nsGDB3PiiSdSf+XHmTNnMnjwYEaOHMk555yzQXvOt9xyC0OHDmX//ffna1/7GgBr1qxh8uTJ7L///gwdOpTvfve7AFx11VXst99+HHDAAUycOHHTX6wN4B611HmMAhZl5mKAiJgGTAAWrKf9JCrhDPDXwD2Z+Wox7z3AOOCWdq1YrfJ3d8xnwUuvt+ky99tjJy7+1JANnq+uro6HHnqILl268Prrr/PAAw/QtWtXZs2axTe+8Q1+/vOfv2eeZ599lvvvv58VK1awzz77cOaZZ77n50tPPPEE8+fPZ4899uDggw/mwQcfpLa2ltNPP505c+YwcOBAJk2a1Oo6X3rpJb72ta/x2GOP0bNnTz7xiU8wffp0+vfvz4svvsjTTz8NwGuvvQbAFVdcwfPPP0+3bt0axm0u7lFLnUdf4IWq4bpi3HtExAeBgcB9GzqvOrcTTjiBLl26ALB8+XJOOOEE9t9/f6ZMmcL8+fObnOeoo46iW7du9O7dm912242XX375PW1GjRpFv3792GabbRg2bBhLlizh2WefZa+99mr4qdOGBPXcuXMZM2YMffr0oWvXrpx44onMmTOHvfbai8WLF3P22Wfzi1/8gp122gmAAw44gBNPPJGf/OQn6z2k317co5bUlInAbZm5ZkNnjIjTgNMA9txzz7auS03YmD3f9rLDDjs03P/Wt77F2LFjuf3221myZAljxoxpcp5u3bo13O/SpQurV6/eqDZtoWfPnjz11FPcfffd/Nu//Ru33norN9xwA3feeSdz5szhjjvu4PLLL+c3v/nNZgts96ilzuNFoH/VcL9iXFMmsu5h7VbPm5nXZWZtZtb26VPqq3CqnS1fvpy+fSsHXm688cY2X/4+++zD4sWLWbJkCQA//elPWz3vqFGj+NWvfsUrr7zCmjVruOWWWzjssMN45ZVXWLt2LccddxyXXXYZjz/+OGvXruWFF15g7NixfOc732H58uW88cYbbf581sc9aqnzmAvsHREDqYTsROBzjRtFxGCgJ/Bw1ei7gb+v6kD2CeDr7VuutnTnn38+J598MpdddhlHHXVUmy//fe97H9dccw3jxo1jhx124MADD1xv23vvvZd+/fo1DP/sZz/jiiuuYOzYsWQmRx11FBMmTOCpp57ii1/8ImvXrgXg29/+NmvWrOGkk05i+fLlZCbnnHMOu+yyS5s/n/WJ+p5zZVFbW5tev1ZqWUQ8lpkt/8Zl3XmOBL5H5edZN2Tm5RFxCTAvM2cUbaYC3TPzgkbzngJ8oxi8PDN/2NLjuT63n2eeeYZ99923o8vocG+88QY9evQgM/nyl7/M3nvvzZQpUzq6rGY19b9rbn12j1rqRDJzJjCz0biLGg1PXc+8NwA3tFtx0kb4/ve/z0033cSqVasYPnw4p59+ekeX1OYMaknSFmvKlCml34PeVHYmkySpxAxqSZJKzKCWJKnEDGpJkkrMoJYkbbCxY8dy9913rzPue9/7HmeeeeZ65xkzZgz1P9c78sgjmzxn9tSpU7nyyiubfezp06ezYMG7p6i/6KKLmDVr1gZU37SyXg7ToJYkbbBJkyYxbdq0dcZNmzat1efbnjlz5kafNKRxUF9yySV8/OMf36hlbQkMaknSBjv++OO58847WbVqFQBLlizhpZde4tBDD+XMM8+ktraWIUOGcPHFFzc5/4ABA3jllVcAuPzyyxk0aBCHHHJIw6UwofIb6QMPPJCamhqOO+44Vq5cyUMPPcSMGTP46le/yrBhw3juueeYPHkyt912G1A5A9nw4cMZOnQop5xyCm+//XbD41188cWMGDGCoUOH8uyzz7b6uXb05TD9HbUkbenuugD+9Ju2XebuQ+GTV6x3cq9evRg1ahR33XUXEyZMYNq0aXzmM58hIrj88svp1asXa9as4fDDD+fXv/41BxxwQJPLeeyxx5g2bRpPPvkkq1evZsSIEYwcORKAY489llNPPRWACy+8kB/84AecffbZjB8/nqOPPprjjz9+nWW99dZbTJ48mXvvvZdBgwbxhS98gWuvvZZzzz0XgN69e/P4449zzTXXcOWVV3L99de3+DKU4XKY7lFLkjZK9eHv6sPet956KyNGjGD48OHMnz9/ncPUjT3wwAMcc8wxbL/99uy0006MHz++YdrTTz/NoYceytChQ7n55pvXe5nMegsXLmTgwIEMGjQIgJNPPpk5c+Y0TD/22GMBGDlyZMOFPFpShsthukctSVu6ZvZ829OECROYMmUKjz/+OCtXrmTkyJE8//zzXHnllcydO5eePXsyefJk3nrrrY1a/uTJk5k+fTo1NTXceOONzJ49e5Pqrb9UZltcJnNzXg7TPWpJ0kbp0aMHY8eO5ZRTTmnYm3799dfZYYcd2HnnnXn55Ze56667ml3GRz/6UaZPn86bb77JihUruOOOOxqmrVixgg984AO888473HzzzQ3jd9xxR1asWPGeZe2zzz4sWbKERYsWAfDjH/+Yww47bJOeYxkuh+ketSRpo02aNIljjjmm4RB4TU0Nw4cPZ/DgwfTv35+DDz642flHjBjBZz/7WWpqathtt93WuVTlpZdeyujRo+nTpw+jR49uCOeJEydy6qmnctVVVzV0IgPo3r07P/zhDznhhBNYvXo1Bx54IGecccYGPZ8yXg7Ty1xKW6iNuczl5ub63H68zOWWa0Mvc+mhb0mSSsygliSpxAxqSZJKzKCWpC1U2foYqWUb8z8zqCVpC9S9e3eWLVtmWG9BMpNly5bRvXv3DZrPn2dJ0haoX79+1NXVsXTp0o4uRRuge/fu6/z8qzUMaknaAm277bYMHDiwo8vQZuChb0mSSsygliSpxAxqSZJKzKCWJKnEDGpJkkrMoJYkqcQMakmSSsygliSpxAxqSZJKzKCWJKnEDGpJkkrMoJYkqcQMakmSSsygliSpxAxqSZJKzKCWJKnEDGpJkkqsVUEdEeMiYmFELIqIC5qYvmdE3B8RT0TEryPiyGL8ERHxWET8pvj7sbZ+ApIkbc26ttQgIroAVwNHAHXA3IiYkZkLqppdCNyamddGxH7ATGAA8Arwqcx8KSL2B+4G+rbxc5AkaavVmj3qUcCizFycmauAacCERm0S2Km4vzPwEkBmPpGZLxXj5wPvi4hum162JEmdQ4t71FT2gF+oGq4DRjdqMxX4ZUScDewAfLyJ5RwHPJ6Zb29EnZIkdUpt1ZlsEnBjZvYDjgR+HBENy46IIcB3gNObmjkiTouIeRExb+nSpW1UkiRJW77WBPWLQP+q4X7FuGp/A9wKkJkPA92B3gAR0Q+4HfhCZj7X1ANk5nWZWZuZtX369NmwZyBJ0lasNUE9F9g7IgZGxHbARGBGozZ/AA4HiIh9qQT10ojYBbgTuCAzH2yzqiVJ6iRaDOrMXA2cRaXH9jNUenfPj4hLImJ80ez/AqdGxFPALcDkzMxivg8BF0XEk8Vtt3Z5JpIkbYVa05mMzJxJ5SdX1eMuqrq/ADi4ifkuAy7bxBolSeq0PDOZJEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS51IRIyLiIURsSgiLlhPm89ExIKImB8R/1E1fk1EPFncZmy+qqXOrWtHFyBp84iILsDVwBFAHTA3ImZk5oKqNnsDXwcOzsy/RMRuVYt4MzOHbc6aJblHLXUmo4BFmbk4M1cB04AJjdqcClydmX8ByMw/b+YaJTViUEudR1/gharhumJctUHAoIh4MCIeiYhxVdO6R8S8Yvyn27lWSQUPfUuq1hXYGxgD9APmRMTQzHwN+GBmvhgRewH3RcRvMvO5xguIiNOA0wD23HPPzVa4tLVyj1rqPF4E+lcN9yvGVasDZmTmO5n5PPBbKsFNZr5Y/F0MzAaGN/UgmXldZtZmZm2fPn3a9hlInZBBLXUec4G9I2JgRGwHTAQa996eTmVvmojoTeVQ+OKI6BkR3arGHwwsQFK789C31Elk5uqIOAu4G+gC3JCZ8yPiEmBeZs4opn0iIhYAa4CvZuayiDgI+PeIWEvlA/4V1b3FJbUfg1rqRDJzJjCz0biLqu4n8JXiVt3mIWDo5qhR0ro89C1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJVYq4I6IsZFxMKIWBQRFzQxfc+IuD8inoiIX0fEkVXTvl7MtzAi/roti5ckaWvXtaUGEdEFuBo4AqgD5kbEjMxcUNXsQuDWzLw2IvYDZgIDivsTgSHAHsCsiBiUmWva+olIkrQ1as0e9ShgUWYuzsxVwDRgQqM2CexU3N8ZeKm4PwGYlplvZ+bzwKJieZIkqRVaE9R9gReqhuuKcdWmAidFRB2VvemzN2BeSZK0Hm3VmWwScGNm9gOOBH4cEa1edkScFhHzImLe0qVL26gkSZK2fK0J0xeB/lXD/Ypx1f4GuBUgMx8GugO9WzkvmXldZtZmZm2fPn1aX70kSVu51gT1XGDviBgYEdtR6Rw2o1GbPwCHA0TEvlSCemnRbmJEdIuIgcDewKNtVbwkSVu7Fnt9Z+bqiDgLuBvoAtyQmfMj4hJgXmbOAP4v8P2ImEKlY9nkzExgfkTcCiwAVgNftse3JEmt12JQA2TmTCqdxKrHXVR1fwFw8HrmvRy4fBNqlCSp0/LMZJIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLUlSiRnUkiSVmEEtSVKJGdSSJJWYQS1JUokZ1JIklZhBLXUiETEuIhZGxKKIuGA9bT4TEQsiYn5E/EfV+JMj4nfF7eTNV7XUuXXt6AIkbR4R0QW4GjgCqAPmRsSMzFxQ1WZv4OvAwZn5l4jYrRjfC7gYqAUSeKyY9y+b+3lInY171FLnMQpYlJmLM3MVMA2Y0KjNqcDV9QGcmX8uxv81cE9mvlpMuwcYt5nqljo1g1rqPPoCL1QN1xXjqg0CBkXEgxHxSESM24B5JbUDD31LqtYV2BsYA/QD5kTE0A1ZQEScBpwGsOeee7Z1fVKn4x611Hm8CPSvGu5XjKtWB8zIzHcy83ngt1SCuzXzApCZ12VmbWbW9unTp82Klzorg1rqPOYCe0fEwIjYDpgIzGjUZjqVvWkiojeVQ+GLgbuBT0REz4joCXyiGCepnXnoW+okMnN1RJxFJWC7ADdk5vyIuASYl5kzeDeQFwBrgK9m5jKAiLiUStgDXJKZr27+ZyF1Pga11Ilk5kxgZqNxF1XdT+Arxa3xvDcAN7R3jZLW5aFvSZJKzKCWJKnEDGpJkkrMoJYkqcQMakmSSsygliSpxAxqSZJKzKCWJKnEDGpJkkrMoJYkqcRaFdQRMS4iFkbEooi4oInp342IJ4vbbyPitapp/xAR8yPimYi4KiKiDeuXJGmr1uK5viOiC3A1cASVS+DNjYgZmbmgvk1mTqlqfzYwvLh/EHAwcEAx+b+Bw4DZbVS/JElbtdbsUY8CFmXm4sxcBUwDJjTTfhJwS3E/ge7AdkA3YFvg5Y0vV5KkzqU1Qd0XeKFquK4Y9x4R8UFgIHAfQGY+DNwP/LG43Z2Zz2xKwZIkdSZt3ZlsInBbZq4BiIgPAfsC/aiE+8ci4tDGM0XEaRExLyLmLV26tI1LkiRpy9WaoH4R6F813K8Y15SJvHvYG+AY4JHMfCMz3wDuAj7SeKbMvC4zazOztk+fPq2rXJKkTqA1QT0X2DsiBkbEdlTCeEbjRhExGOgJPFw1+g/AYRHRNSK2pdKRzEPfkiS1UotBnZmrgbOAu6mE7K2ZOT8iLomI8VVNJwLTMjOrxt0GPAf8BngKeCoz72iz6iVJ2sq1+PMsgMycCcxsNO6iRsNTm5hvDXD6JtQnSVKn5pnJJEkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1rqRCJiXEQsjIhFEXFBE9MnR8TSiHiyuH2patqaqvEzNm/lUufVtaMLkLR5REQX4GrgCKAOmBsRMzJzQaOmP83Ms5pYxJuZOaydy5TUiHvUUucxCliUmYszcxUwDZjQwTVJaoFBLXUefYEXqobrinGNHRcRv46I2yKif9X47hExLyIeiYhPt2ehkt5lUEuqdgcwIDMPAO4Bbqqa9sHMrAU+B3wvIv6qqQVExGlFoM9bunRp+1csbeVaFdSt6IDy3apOJr+NiNeqpu0ZEb+MiGciYkFEDGi78iVtgBeB6j3kfsW4Bpm5LDPfLgavB0ZWTXux+LsYmA0Mb+pBMvO6zKzNzNo+ffq0XfVSJ9ViUFd1QPkksB8wKSL2q26TmVMyc1jR0eRfgP+smvwj4B8zc18q35H9uY1ql7Rh5gJ7R8TAiNgOmAis03s7Ij5QNTgeeKYY3zMiuhX3ewMHA407oUlqB63p9d3QAQUgIuo7oKxvJZ0EXFy03Q/ompn3AGTmG5tcsaSNkpmrI+Is4G6gC3BDZs6PiEuAeZk5AzgnIsYDq4FXgcnF7PsC/x4Ra6l8wL+iid7iktpBa4K6qQ4oo5tqGBEfBAYC9xWjBgGvRcR/FuNnARdk5pqNrljSRsvMmcDMRuMuqrr/deDrTcz3EDC03QuU9B5t3ZlsInBbVRB3BQ4FzgMOBPbi3U/oDex8IklS01oT1C12QKkyEbilargOeLL43eZqYDowovFMdj6RJKlprQnqFjugAETEYKAn8HCjeXeJiPr0/Rh2QJEkqdVaDOpiT7i+A8ozwK31HVCKTif1JgLTMjOr5l1D5bD3vRHxGyCA77flE5AkaWvWqnN9t9QBpRieup557wEO2Mj6JEnq1DwzmSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYq0K6ogYFxELI2JRRFzQxPTvRsSTxe23EfFao+k7RURdRPxrG9UtSVKn0LWlBhHRBbgaOAKoA+ZGxIzMXFDfJjOnVLU/GxjeaDGXAnPapGJJkjqR1uxRjwIWZebizFwFTAMmNNN+EnBL/UBEjATeD/xyUwqVJKkzak1Q9wVeqBquK8a9R0R8EBgI3FcMbwP8E3DeppUpSVLn1NadySYCt2XmmmL4b4GZmVnX3EwRcVpEzIuIeUuXLm3jkiRJ2nK1+B018CLQv2q4XzGuKROBL1cNfwQ4NCL+FugBbBcRb2TmOh3SMvM64DqA2trabGXtkiRt9VoT1HOBvSNiIJWAngh8rnGjiBgM9AQerh+XmSdWTZ8M1DYOaUmStH4tHvrOzNXAWcDdwDPArZk5PyIuiYjxVU0nAtMy0z1iSZLaSGv2qMnMmcDMRuMuajQ8tYVl3AjcuEHVSZLUyXlmMqkTacXJiyZHxNKqExh9qWrayRHxu+J28uatXOq8WrVHLWnL15qTFxV+mplnNZq3F3AxUAsk8Fgx7182Q+lSp+YetdR5bOjJi6r9NXBPZr5ahPM9wLh2qlNSFYNa6jxae/Ki4yLi1xFxW0TU/zRzQ0585HkRpDZkUEuqdgcwIDMPoLLXfNOGLiAzr8vM2sys7dOnT5sXKHU2BrXUebR48qLMXJaZbxeD1wMjWzuvpPZhUEudR8PJiyJiOyrnPphR3SAiPlA1OJ7KuROgch6FT0REz4joCXyiGCepndnrW+okMnN1RNSfvKgLcEP9yYuAeZk5AzinOJHRauBVYHIx76sRcSmVsAe4JDNf3exPQuqEDGqpE2np5EWZ+XXg6+uZ9wbghnYtUNJ7eOhbkqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEjOoJUkqMYNakqQSM6glSSoxg1qSpBIzqCVJKjGDWpKkEmtVUEfEuIhYGBGLIuKCJqZ/NyKeLG6/jYjXivHDIuLhiJgfEb+OiM+2cf2SJG3VurbUICK6AFcDRwB1wNyImJGZC+rbZOaUqvZnA8OLwZXAFzLzdxGxB/BYRNydma+14XOQJGmr1Zo96lHAosxcnJmrgGnAhGbaTwJuAcjM32bm74r7LwF/BvpsWsmSJHUerQnqvsALVcN1xbj3iIgPAgOB+5qYNgrYDnhuw8uUJKlzauvOZBOB2zJzTfXIiPgA8GPgi5m5tvFMEXFaRMyLiHlLly5t45IkSdpytSaoXwT6Vw33K8Y1ZSLFYe96EbETcCfwzcx8pKmZMvO6zKzNzNo+fTwyLklSvdYE9Vxg74gYGBHbUQnjGY0bRcRgoCfwcNW47YDbgR9l5m1tU7IkSZ1Hi0GdmauBs4C7gWeAWzNzfkRcEhHjq5pOBKZlZlaN+wzwUWBy1c+3hrVd+ZIkbd1a/HkWQGbOBGY2GndRo+GpTcz3E+Anm1CfJEmdmmcmkySpxAxqSZJKzKCWJKnEDGpJkkrMoJYkqcQMakmSSqxVP8/S5vPOO+9QV1fHW2+91dGlqCS6d+9Ov3792HbbbTu6FEkdwKAumbq6OnbccUcGDBhARHR0OepgmcmyZcuoq6tj4MCBHV2OpA7goe+Seeutt9h1110NaQEQEey6664eYZE6MYO6hAxpVfP9IHVuBrXWsWzZMoYNG8awYcPYfffd6du3b8PwqlWrmp133rx5nHPOOS0+xkEHHdRW5QJw7rnn0rdvX9aufc8VVCVpi+d31FrHrrvuypNPPgnA1KlT6dGjB+edd17D9NWrV9O1a9Nvm9raWmpra1t8jIceeqhNagVYu3Ytt99+O/379+dXv/oVY8eObbNlV2vueUtSe3KPWi2aPHkyZ5xxBqNHj+b888/n0Ucf5SMf+QjDhw/noIMOYuHChQDMnj2bo48+GqiE/CmnnMKYMWPYa6+9uOqqqxqW16NHj4b2Y8aM4fjjj2fw4MGceOKJ1F98bebMmQwePJiRI0dyzjnnNCy3sdmzZzNkyBDOPPNMbrnl3Uuhv/zyyxxzzDHU1NRQU1PT8OHgRz/6EQcccAA1NTV8/vOfb3h+t9327lVYq+s79NBDGT9+PPvttx8An/70pxk5ciRDhgzhuuuua5jnF7/4BSNGjKCmpobDDz+ctWvXsvfee7N06VKg8oHiQx/6UMOwJLWWuwgl9nd3zGfBS6+36TL322MnLv7UkA2er66ujoceeoguXbrw+uuv88ADD9C1a1dmzZrFN77xDX7+85+/Z55nn32W+++/nxUrVrDPPvtw5plnvucnRk888QTz589njz324OCDD+bBBx+ktraW008/nTlz5jBw4EAmTZq03rpuueUWJk2axIQJE/jGN77BO++8w7bbbss555zDYYcdxu23386aNWt44403mD9/PpdddhkPPfQQvXv35tVXX23xeT/++OM8/fTTDT2ub7jhBnr16sWbb77JgQceyHHHHcfatWs59dRTG+p99dVX2WabbTjppJO4+eabOffcc5k1axY1NTX06dNnA195SZ2de9RqlRNOOIEuXboAsHz5ck444QT2339/pkyZwvz585uc56ijjqJbt2707t2b3XbbjZdffvk9bUaNGkW/fv3YZpttGDZsGEuWLOHZZ59lr732agjH9QX1qlWrmDlzJp/+9KfZaaedGD16NHfffTcA9913H2eeeSYAXbp0Yeedd+a+++7jhBNOoHfv3gD06tWrxec9atSodX4WddVVV1FTU8OHP/xhXnjhBX73u9/xyCOP8NGPfrShXf1yTznlFH70ox8BlYD/4he/2OLjtbeIGBcRCyNiUURc0Ey74yIiI6K2GB4QEW9WXVf+3zZf1VLn5h51iW3Mnm972WGHHRruf+tb32Ls2LHcfvvtLFmyhDFjxjQ5T7du3Rrud+nShdWrV29Um/W5++67ee211xg6dCgAK1eu5H3ve996D5OvT9euXRs6oq1du3adTnPVz3v27NnMmjWLhx9+mO23354xY8Y0+7Op/v378/73v5/77ruPRx99lJtvvnmD6mprEdEFuBo4AqgD5kbEjMxc0KjdjsD/Af6n0SKey8xhm6NWSe9yj1obbPny5fTt2xeAG2+8sc2Xv88++7B48WKWLFkCwE9/+tMm291yyy1cf/31LFmyhCVLlvD8889zzz33sHLlSg4//HCuvfZaANasWcPy5cv52Mc+xs9+9jOWLVsG0HDoe8CAATz22GMAzJgxg3feeafJx1u+fDk9e/Zk++2359lnn+WRRx4B4MMf/jBz5szh+eefX2e5AF/60pc46aST1jki0YFGAYsyc3FmrgKmAROaaHcp8B3AH29LJWBQa4Odf/75fP3rX2f48OEbtAfcWu973/u45pprGDduHCNHjmTHHXdk5513XqfNypUr+cUvfsFRRx3VMG6HHXbgkEMO4Y477uCf//mfuf/++xk6dCgjR45kwYIFDBkyhG9+85scdthh1NTU8JWvfAWAU089lV/96lfU1NTw8MMPr7MXXW3cuHGsXr2afffdlwsuuIAPf/jDAPTp04frrruOY489lpqaGj772c82zDN+/HjeeOONUhz2BvoCL1QN1xXjGkTECKB/Zt7ZxPwDI+KJiPhVRBy6vgeJiNMiYl5EzLPznLTpor6XbVnU1tbmvHnzOrqMDvPMM8+w7777dnQZHe6NN96gR48eZCZf/vKX2XvvvZkyZUpHl7XB5s2bx5QpU3jggQc2aTlNvS8i4rHMbPn3cO+2Px4Yl5lfKoY/D4zOzLOK4W2A+4DJmbkkImYD52XmvIjoBvTIzGURMRKYDgzJzGZ7O3b29VlqrebWZ/eoVUrf//73GTZsGEOGDGH58uWcfvrpHV3SBrviiis47rjj+Pa3v93RpdR7EehfNdyvGFdvR2B/YHZELAE+DMyIiNrMfDszlwFk5mPAc8CgzVK11Mm5R10y7lGrKW20R90V+C1wOJWAngt8LjOb7LbfaI+6D/BqZq6JiL2AB4Chmdnsb9w6+/ostVZz67O9vqVOIjNXR8RZwN1AF+CGzJwfEZcA8zJzRjOzfxS4JCLeAdYCZ7QU0pLahkEtdSKZOROY2WjcRetpO6bq/s+B957VRlK78ztqSZJKzKCWJKnEDGqtY+zYsQ2n4az3ve99r+F0nE0ZM2YM9R2GjjzySF577bX3tJk6dSpXXnlls489ffp0Fix49yRZF110EbNmzdqA6pvn5TAlbYkMaq1j0qRJTJs2bZ1x06ZNa/bCGNVmzpzJLrvsslGP3TioL7nkEj7+8Y9v1LIaa3w5zPbSHieAkdS5GdRax/HHH8+dd97ZcL7rJUuW8NJLL3HooYdy5plnUltby5AhQ7j44oubnH/AgAG88sorAFx++eUMGjSIQw45pOFSmFD5jfSBBx5ITU0Nxx13HCtXruShhx5ixowZfPWrX2XYsGE899xz61x+8t5772X48OEMHTqUU045hbfffrvh8S6++GJGjBjB0KFDefbZZ5usy8thStpS2eu7zO66AP70m7Zd5u5D4ZNXrHdyr169GDVqFHfddRcTJkxg2rRpfOYznyEiuPzyy+nVqxdr1qzh8MMP59e//jUHHHBAk8t57LHHmDZtGk8++SSrV69mxIgRjBw5EoBjjz2WU089FYALL7yQH/zgB5x99tmMHz+eo48+muOPP36dZb311ltMnjyZe++9l0GDBvGFL3yBa6+9lnPPPReA3r178/jjj3PNNddw5ZVXcv3117+nHi+HKWlL5R613qP68Hf1Ye9bb72VESNGMHz4cObPn7/OYerGHnjgAY455hi23357dtppJ8aPH98w7emnn+bQQw9l6NCh3Hzzzeu9TGa9hQsXMnDgQAYNqpwI6+STT2bOnDkN04899lgARo4c2XAhj2peDlPSlsw96jJrZs+3PU2YMIEpU6bw+OOPs3LlSkaOHMnzzz/PlVdeydy5c+nZsyeTJ09u9hKPzZk8eTLTp0+npqaGG2+8kdmzZ29SvfWXylzfZTK9HKakLZl71HqPHj16MHbsWE455ZSGvenXX3+dHXbYgZ133pmXX36Zu+66q9llfPSjH2X69Om8+eabrFixgjvuuKNh2ooVK/jABz7AO++8s04o7bjjjqxYseI9y9pnn31YsmQJixYtAuDHP/4xhx12WKufj5fDlLQlM6jVpEmTJvHUU081BHVNTQ3Dhw9n8ODBfO5zn+Pggw9udv4RI0bw2c9+lpqaGj75yU9y4IEHNky79NJLGT16NAcffDCDBw9uGD9x4kT+8R//keHDh/Pcc881jO/evTs//OEPOeGEExg6dCjbbLMNZ5xxRqueh5fDlLSl86IcJeNFOTqnli6H2RYX5egInX19llrLi3JIJXbFFVdw7bXX+t20pCZ56FvqYBdccAG///3vOeSQQzq6FEklZFBLklRiBnUJla3fgDqW7wepczOoS6Z79+4sW7bMjbOASkgvW7aM7t27d3QpkjqInclKpl+/ftTV1XmuZzXo3r07/fr16+gyJHUQg7pktt1223VORSlJ6tw89C1JUokZ1JIklZhBLUlSiZXuFKIRsRT4/WZ8yN7AK5vx8VrDmlqns9f0wcws9YWrXZ+B8tVUtnrAmqCZ9bl0Qb25RcS8sp0v2Zpax5rUWBlf/7LVVLZ6wJpa4qFvSZJKzKCWJKnEDGq4rqMLaII1tY41qbEyvv5lq6ls9YA1NavTf0ctSVKZuUctSVKJdYqgjoheEXFPRPyu+NtzPe1OLtr8LiJObmL6jIh4uqNriojtI+LOiHg2IuZHxBWbWMu4iFgYEYsi4oImpneLiJ8W0/8nIgZUTft6MX5hRPz1ptSxqfVExBER8VhE/Kb4+7G2qGdTaqqavmdEvBER57VVTZ1V2dZn1+X2qcn1uUpmbvU34B+AC4r7FwDfaaJNL2Bx8bdncb9n1fRjgf8Anu7omoDtgbFFm+2AB4BPbmQdXYDngL2KZT0F7Neozd8C/1bcnwj8tLi/X9G+GzCwWE6XTXxdNqWe4cAexf39gRfb6H+10TVVTb8N+BlwXkevD1v6rWzrs+tyu9Xk+lzcOsUeNTABuKm4fxPw6Sba/DVwT2a+mpl/Ae4BxgFERA/gK8BlZagpM1dm5v0AmbkKeBzY2MsrjQIWZebiYlnTitrWV+ttwOEREcX4aZn5dmY+DywqlrcpNrqezHwiM18qxs8H3hcR3Taxnk2qCSAiPg08X9SkTVe29dl1uR1qcn1+V2cJ6vdn5h+L+38C3t9Em77AC1XDdcU4gEuBfwJWlqgmACJiF+BTwL0bWUeLj1HdJjNXA8uBXVs57+asp9pxwOOZ+fYm1rNJNRWh8DXg79qgDlWUbX12XW6fmqp16vV5q7nMZUTMAnZvYtI3qwcyMyOi1V3dI2IY8FeZOaXx9xQdVVPV8rsCtwBXZebiDZ1/axURQ4DvAJ/o6FqAqcB3M/ON4gO5WqFs67Prcsdxfd6KgjozP76+aRHxckR8IDP/GBEfAP7cRLMXgTFVw/2A2cBHgNqIWELl9dotImZn5hha0I411bsO+F1mfq+lWprxItC/0WO8uJ42dcUGZWdgWSvn3Zz1EBH9gNuBL2Tmc5tYS1vUNBo4PiL+AdgFWBsRb2Xmv7ZRbVulsq3PrssdUpPrc73N8UV4R9+Af2Tdzh7/0ESbXlS+d+hZ3J4HejVqM4C260y2STVR+X7t58A2m1hHVyodWwbybseKIY3afJl1O1bcWtwfwrodUBaz6Z3JNqWeXYr2x7bx+2eja2rUZip2JmuL/0ep1mfX5XaryfW5/rE2x4N09I3K9x33Ar8DZlWtILXA9VXtTqHSiWIR8MUmltMmK/am1kTlE2ACzwBPFrcvbUItRwK/pdIT8pvFuEuA8cX97lR6OC4CHgX2qpr3m8V8C9nI3qptVQ9wIfC/Va/Jk8BuHVlTo2VsthV7a76VbX12XW6fmlyf3715ZjJJkkqss/T6liRpi2RQS5JUYga1JEklZlBLklRiBrUkSSVmUEuSVGIGtSRJJWZQS5JUYv8/RFXiitaAgBMAAAAASUVORK5CYII=
    pyplot.plot(epochs_range, loss, label='Training Loss')
    pyplot.plot(epochs_range, val_loss, label='Validation Loss')
    pyplot.legend(loc='upper right')
    pyplot.title(name+': Training and Validation Loss')
    pyplot.show()

SyntaxError: invalid syntax (<ipython-input-13-6bd62539e857>, line 18)

In [ ]:
class_names = ["Wearing Mask a Correctly Over Mouth and Nose", "Wearing Mask a Over Mouth, but not Over Nose", "Not Wearing a Mask Over Mount and Nose", "Not Wearing a Mask"]

In [ ]:
def predict(file_path):
    
    img = tf.keras.utils.load_img(file_path, target_size=(150, 150))
    pyplot.imshow(img)

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    for name in models:
        model = models[name]
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])


        print(
            "{} predicts {} ({:.2f}% confidence)"
            .format(name, class_names[np.argmax(score)], 100 * np.max(score))
        )
    pyplot.show()

In [ ]:
predict("faces/benjamin/1.jpg")

In [ ]:
predict("faces/benjamin/2.jpg")

In [ ]:
predict("faces/benjamin/3.jpg")

In [ ]:
predict("faces/benjamin/4.jpg")

In [ ]:
predict("faces/shaurya/1.jpg")

In [ ]:
predict("faces/shaurya/2.jpg")

In [ ]:
predict("faces/shaurya/3.jpg")

In [ ]:
predict("faces/shaurya/4.jpg")

In [ ]:
predict("faces/zunyi/1.jpg")

In [ ]:
predict("faces/zunyi/2.jpg")

In [ ]:
predict("faces/zunyi/3.jpg")

In [ ]:
predict("faces/zunyi/4.jpg")

In [ ]:
predict("faces/parisa/1.jpg")

In [ ]:
predict("faces/parisa/2.jpg")

In [ ]:
predict("faces/parisa/3.jpg")

## Source: https://www.tensorflow.org/tutorials/images/classification